In [25]:
from datasets import load_dataset
from transformers import pipeline

In [3]:
poem_dataset = load_dataset(
    "poem_sentiment",
    version = "1.0.0",
    cache_dir='/home/jellyfish/llms/databricks/data'
)


/home/jellyfish/llms/simienv/lib/python3.11/site-packages/datasets/load.py:1461: FutureWarning: The repository for poem_sentiment contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/poem_sentiment
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Generating test split: 100%|██████████| 104/104 [00:00<00:00, 33290.67 examples/s]


In [7]:
poem_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'verse_text', 'label'],
        num_rows: 892
    })
    validation: Dataset({
        features: ['id', 'verse_text', 'label'],
        num_rows: 105
    })
    test: Dataset({
        features: ['id', 'verse_text', 'label'],
        num_rows: 104
    })
})

In [8]:
poem_dataset['train']

Dataset({
    features: ['id', 'verse_text', 'label'],
    num_rows: 892
})

In [11]:
range(0,10)

range(0, 10)

In [22]:
poem_dataset['train'].select([0,1,2,3])

Dataset({
    features: ['id', 'verse_text', 'label'],
    num_rows: 4
})

In [23]:
poem_sample = poem_dataset['train'].select(range(10))

In [24]:
poem_sample.to_pandas()

,id,verse_text,label
0,0,with pale blue berries. in these peaceful shad...,1
1,1,"it flows so long as falls the rain,",2
2,2,"and that is why, the lonesome day,",0
3,3,"when i peruse the conquered fame of heroes, an...",3
4,4,of inward strife for truth and liberty.,3
5,5,the red sword sealed their vows!,3
6,6,and very venus of a pipe.,2
7,7,"who the man, who, called a brother.",2
8,8,"and so on. then a worthless gaud or two,",0
9,9,to hide the orb of truth--and every throne,2


In [27]:
sentiment_classifier = pipeline(
    task='text-classification',
    model='nickwong64/bert-base-uncased-poems-sentiment',
    model_kwargs={'cache_dir':'/home/jellyfish/llms/databricks/data'}
)

In [28]:
results = sentiment_classifier(poem_sample['verse_text'])

In [29]:
results

[{'label': 'positive', 'score': 0.9965937733650208},
 {'label': 'no_impact', 'score': 0.9987409710884094},
 {'label': 'negative', 'score': 0.995965838432312},
 {'label': 'mixed', 'score': 0.9687354564666748},
 {'label': 'mixed', 'score': 0.9759674668312073},
 {'label': 'mixed', 'score': 0.9665797352790833},
 {'label': 'no_impact', 'score': 0.9986388087272644},
 {'label': 'no_impact', 'score': 0.9986108541488647},
 {'label': 'negative', 'score': 0.9965572357177734},
 {'label': 'no_impact', 'score': 0.9985186457633972}]

In [31]:
import pandas as pd 
joined_data = pd.DataFrame.from_dict(results).rename({"label":"predicted_label"},axis=1).join(
    pd.DataFrame.from_dict(poem_sample).rename({"label":"true_label"},axis=1)
)

sentiment_labels = {0:"negative",1:"positive",2:"no_impact",3:"mixed"}
joined_data = joined_data.replace({"true_label":sentiment_labels})

joined_data[['predicted_label','true_label','score','verse_text']]


,predicted_label,true_label,score,verse_text
0,positive,positive,0.996594,with pale blue berries. in these peaceful shad...
1,no_impact,no_impact,0.998741,"it flows so long as falls the rain,"
2,negative,negative,0.995966,"and that is why, the lonesome day,"
3,mixed,mixed,0.968735,"when i peruse the conquered fame of heroes, an..."
4,mixed,mixed,0.975967,of inward strife for truth and liberty.
5,mixed,mixed,0.966580,the red sword sealed their vows!
6,no_impact,no_impact,0.998639,and very venus of a pipe.
7,no_impact,no_impact,0.998611,"who the man, who, called a brother."
8,negative,negative,0.996557,"and so on. then a worthless gaud or two,"
9,no_impact,no_impact,0.998519,to hide the orb of truth--and every throne
